In [ ]:
import keras
import numpy as np
from keras import layers
import random
import sys

'''Реализация посимвольной генерации текста на основе LSTM.'''

# Подготовка данных
# Загрузка и парсинг исходного текстового файла
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

# Векторизация последовательности символов
maxlen = 60 # Извлечение последовательностей по 60 символов

step = 3 # Новые последовательности выбираются из через каждые 3 символа

sentences = [] # Хранение извлеченных последовательностей

next_chars = [] # Хранение целей (символов, следующих за последовательностями)

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text))) # Список уникальных символов в корпусе
print('Unique characters:', len(chars))
# Словарь, отображающий уникальные символы в их индексы в списке "chars"
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    # Прямое кодирование символов в бинарные массивы
    y[i, char_indices[next_chars[i]]] = 1

# Конструирование сети

# Модель с единственным слоем LSTM для предсказания следующего символа
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

# Конфигурация компилируемой модели
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Обучение модели и извлечение образцов их нее

# Функция выборки следющего символа с учетом прогнозов модели
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Цикл генерации текста

for epoch in range(1, 60): # Обучение модели в течение 60 эпох
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1) # Выполнение одной генерации обуч
    # Выбор случайного начального текста
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    # Генерация текста для разных температур
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400): # Генерация 400 символов, начиная с нач-го текста
            # Прямое кодирование символов, сгенерированных до сих пор
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # Выбор следующего символа
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
			